In [45]:
import sqlite3
from django.contrib.auth.hashers import make_password
from django.conf import settings
import pandas as pd
import numpy as np
from faker import Faker
import random
from sqlalchemy import create_engine
from datetime import datetime

settings.configure()

In [108]:
connection = sqlite3.connect('db.sqlite3')
cursor = connection.cursor()

In [ ]:
cursor.execute('SELECT name FROM sqlite_master WHERE type="table";')
print(cursor.fetchall())

In [ ]:
cursor.execute('PRAGMA table_info([user_customuser]);')
print(cursor.fetchall())

In [3]:
# Por motivos desconhecidos, o Django usa o nome errado e diz que a coluna não existe, então alteramos

cursor.execute('ALTER TABLE user_business_invitations RENAME COLUMN from_business_id TO business_id;')

In [124]:
fake = Faker('pt_BR')

ddd = [11,12,13,14,15,16,17,18,19,21,22,24,27,28,31,32,33,34,35,37,38,41,42,43,44,45,46,47,48,49,51,53,54,55,61,62,63,64,65,66,67,68,69,71,73,74,75,77,79,81,82,83,84,85,86,87,88,89,91,92,93,94,95,96,97,98,99]
ddd = np.array(ddd, dtype=str)

def create_rows(num=1):
    output = [{"password": make_password(fake.password()),
               "email": fake.unique.email(),
               "first_name": fake.first_name(),
               "last_name": fake.last_name(),
               "is_active": True,
               "is_admin": False,
               "last_login": fake.date_time_between(start_date='-1M', end_date='now'),
               "business_id": random.randint(1,205)
            } for x in range(num)]
    n = int(num/2)
    phone = np.array([random.randint(900000000, 999999999) for x in range(n)], dtype=str)
    result = pd.DataFrame(output)
    result['cpf'] = np.append(np.array(random.sample(range(10000000000, 20000000000), n)), np.array([None]*n))
    result['phone_number'] = np.append(np.array([None]*n), list(map(lambda x: random.choice(ddd)+x, phone)))
    result['birth_date'] = np.append([fake.date_between(start_date='-70y', end_date='-18y') for x in range(n)], np.array([None]*n))
    result['is_client'] = np.append(np.array([False]*n), np.array([True]*n))
    return result

df = create_rows(2000)
df.head()

,password,email,first_name,last_name,is_active,is_admin,last_login,business,cpf,phone_number,birth_date,is_client
0,pbkdf2_sha256$260000$0XZ2pky2T80U3R82yZb1zB$D2...,lucas-gabrielda-mata@example.com,Ana Clara,da Mata,True,False,2021-11-20 20:24:52,100,13719911719,None,1993-09-21,False
1,pbkdf2_sha256$260000$53eWnCz39ueFJ0G56PLbvC$AN...,sarahazevedo@example.com,Miguel,Farias,True,False,2021-11-26 23:08:31,7,18343157727,None,1998-03-01,False
2,pbkdf2_sha256$260000$jdTHXYYoUtxb3Qoyv5SiAF$sT...,silveiralorena@example.com,Emanuella,Ferreira,True,False,2021-12-03 17:17:34,6,17077519600,None,1956-10-30,False
3,pbkdf2_sha256$260000$xXCPZfDMUnWGl5RGINUgoW$zs...,hsouza@example.net,Lucca,Barros,True,False,2021-11-26 07:18:45,176,14024263800,None,1968-06-03,False
4,pbkdf2_sha256$260000$KvshA3YwhjgzSFyTH92tkt$RW...,valentina69@example.com,Eloah,Freitas,True,False,2021-12-02 16:22:48,96,11118031212,None,1976-10-12,False


In [127]:
engine = create_engine('sqlite:///db.sqlite3')
df.to_sql(name = 'user_customuser', con = engine, if_exists = 'append', index = False)

In [111]:
connection.commit()
cursor.close()
connection.close()